In [38]:
from empath import Empath
from empath import helpers as util
import pandas as pd
import os

categories = Empath()
cats = categories.analyze("Testing", normalize=False)

In [ ]:
directory = 'Taylor'
files = []
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    if os.path.isfile(f):
        files.append(f)
        
print (files)

for i in range(len(files)):
    read_df = pd.read_csv(files[i])
    if i > 0:
        df = pd.concat([df, read_df], ignore_index=True)
    else: 
        df = read_df
print(df)

In [1]:
#df.to_csv("taylordatawithlinenums.csv")

['Taylor\\01-taylor_swift.csv', 'Taylor\\02-fearless_taylors_version.csv', 'Taylor\\03-speak_now_deluxe_package.csv', 'Taylor\\04-red_deluxe_edition.csv', 'Taylor\\05-1989_deluxe.csv', 'Taylor\\06-reputation.csv', 'Taylor\\07-lover.csv', 'Taylor\\08-folklore_deluxe_version.csv', 'Taylor\\09-evermore_deluxe_version.csv']
                     album_name       track_title  track_n  \
0                  Taylor Swift        Tim McGraw        1   
1                  Taylor Swift        Tim McGraw        1   
2                  Taylor Swift        Tim McGraw        1   
3                  Taylor Swift        Tim McGraw        1   
4                  Taylor Swift        Tim McGraw        1   
...                         ...               ...      ...   
8353  evermore (deluxe version)  ​it’s time to go       17   
8354  evermore (deluxe version)  ​it’s time to go       17   
8355  evermore (deluxe version)  ​it’s time to go       17   
8356  evermore (deluxe version)  ​it’s time to go       17

In [48]:
import pandas as pd
#df = pd.read_csv("spotify.csv")

ldf = pd.read_csv("ArianaGrande.csv")

print(df.shape)
ldf = ldf[['Title','Lyric']]
print(ldf.shape)
# print("ldf:")
# ldf.head(2)

# print(ldf.Album.unique())
# print(df.album_name.unique())
albs = list(df.track_name.unique())

#dff = ldf[['Album', 'Title', "Lyric"]]
#print(dff.head())
ldf.drop(ldf.index[ldf["Title"].isin(albs) == False], inplace = True)
print(ldf.shape)
ldf.head()


(269, 22)
(479, 2)
(105, 2)


,Title,Lyric
2,Lover,we could leave the christmas lights up 'til ja...
4,Look What You Made Me Do,i don't like your little games don't like your...
6,End Game,taylor swift future i wanna be your end game ...
7,ME!,taylor swift i promise that you'll never find ...
8,You Need To Calm Down,you are somebody that i don't know but you're ...


In [49]:
x = 0

print(ldf.loc[ldf["Title"]=="Picture To Burn"].values)
print(ldf.loc[ldf["Title"]=="Picture To Burn",'Lyric'].values)

for i in df['track_name']:
    lyr = ldf.loc[ldf.Title==str(i),'Lyric'].values.tolist()
   # print(type(lyr))
    df.loc[df.track_name==str(i),'lyric'] = str(lyr)
    if len(str(lyr)) > 3:
       # print(str(lyr))
        x+=1
print(x)
df = df[df.lyric != '[]']
#df = df.dropna(subset = ["lyric"]) 
print(df.shape)
df.head()
print(list(df.columns))

[]
[]
105
(105, 23)
['Unnamed: 0', 'album_name', 'track_name', 'artist', 'featuring', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature', 'duration_ms', 'explicit', 'key_name', 'mode_name', 'key_mode', 'lyric']


In [50]:
#get category names
mastercats = pd.read_csv('categ.csv')
mastercats.columns = ['category']
mastercats.at[193, 'category'] = 'help'
mcs = []
for i in range(0, mastercats.shape[0]):
    mcs.append(mastercats.at[i, 'category'])

df[mcs] = ''

C:\Users\lilyg\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\frame.py:3645: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[col] = value


In [52]:
df_cols = ['category', 'count','albums', 'songs']
catdf = pd.DataFrame(columns=df_cols)
catdf.empty

#iterate over each song entry
for i in range(0, df.shape[0]):
    
    lyrList = []
    index = df.iloc[i].name
    lyrList = df.at[index, 'lyric']
    cats = categories.analyze(str(lyrList), normalize=False)
    catList = []
    thisAlb = df.at[index, 'album_name']
    thisSong = df.at[index, 'track_name']

    for key, value in cats.items():
        
        #if the category was detected
        if (value > 0.0):
            #add category and value to category list per song    
            catList.append([str(key),value])
            
            if str(key) in catdf['category'].values:
                
                #get current value to update
                temp = catdf.at[str(key), 'count']
                albs = catdf.at[str(key), 'albums']
                songs = catdf.at[str(key), 'songs']
                songs.update({thisSong: value})  
               
                #update value per album
                if str(thisAlb) in str(albs):
                    catdf.at[str(key), 'count'] = value + temp
                else: 
                    albs.update({thisAlb: value})
                    catdf.loc[str(key)] = [str(key),value + temp, albs,songs]  
            else: 
                songs = {thisSong : value}
                catdf.loc[str(key)] = [str(key),value, {thisAlb : value},songs]  
    
       # df.at[index,'catList'] = catList

        df.at[index, str(key)] = value
        
#print(df.head(20))  
#print(catdf.head(20))
df = df[['album_name', 'track_name', 'artist', 'featuring', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature', 'duration_ms', 'explicit', 'key_name', 'mode_name', 'key_mode', 'lyric', 'negative_emotion', 'positive_emotion', 'friends', 'children', 'speaking', 'optimism', 'family', 'love', 'pain', 'violence', 'hate']]
df.to_csv("spotifycats.csv")

C:\Users\lilyg\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\indexing.py:1684: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self.obj[key] = infer_fill_value(value)


In [4]:
df_cols = ['category', 'count','albums', 'songs']
catdf = pd.DataFrame(columns=df_cols)
catdf.empty

#iterate over each song entry
for i in range(0, df.shape[0]):
    
    lyrList = []
    index = df.iloc[i].name
    lyrList = df.at[index, 'lyric']
    cats = categories.analyze(str(lyrList), normalize=False)
    catList = []
    thisAlb = df.at[index, 'album_name']
    thisSong = df.at[index, 'track_title']

    for key, value in cats.items():
        
        #if the category was detected
        if (value > 0.0):
            #add category and value to category list per song    
            catList.append([str(key),value])
            
            if str(key) in catdf['category'].values:
                
                #get current value to update
                temp = catdf.at[str(key), 'count']
                albs = catdf.at[str(key), 'albums']
                songs = catdf.at[str(key), 'songs']
                songs.update({thisSong: value})  
               
                #update value per album
                if str(thisAlb) in str(albs):
                    catdf.at[str(key), 'count'] = value + temp
                else: 
                    albs.update({thisAlb: value})
                    catdf.loc[str(key)] = [str(key),value + temp, albs,songs]  
            else: 
                songs = {thisSong : value}
                catdf.loc[str(key)] = [str(key),value, {thisAlb : value},songs]  
    
       # df.at[index,'catList'] = catList

        df.at[index, str(key)] = value
        
#print(df.head(20))  
#print(catdf.head(20))
#df.to_csv("taylorcatswithlinenums.csv")

KeyError: 'lyric'

In [44]:
topten = catdf['count'].nlargest(n=10)
ttlist = ["album_name","track_name","track_n","lyric","line"]

ttind = list(topten.index)
for i in range(len(ttind)):
    ttlist.append(ttind[i])

print(ttlist)    
#topten.plot(kind='pie', labels= None, autopct="%1.1f%%", legend = True, title = "Top Ten Categories in Taylor Swift Songs", figsize=(8,5))

['album_name', 'track_name', 'track_n', 'lyric', 'line', 'negative_emotion', 'positive_emotion', 'friends', 'children', 'speaking', 'optimism', 'family', 'love', 'pain', 'violence']


In [5]:
import nltk
from nltk import word_tokenize
#nltk.download('averaged_perceptron_tagger')
#ldf = df.groupby('track_title')
ldf = df
ldf = ldf[ttlist]
ldf["Lyric Struct"] = ''
ldf["LS"] = ''
ldf["Element"] = ''
ldf["e"] = ""

#get unique song titles
unq = ldf.track_title.unique()
patterns = {0: "AABA", 1: "ABAB", 2: "ABABCB", 3: "ABA"}
    
for i in range(ldf.shape[0]):
    index = ldf.iloc[i].name
    lyr = str(ldf.loc[index, "lyric"])

    ldf.loc[index, "lyric"] = lyr

    text = word_tokenize(str(lyr))
    t = nltk.pos_tag(text)
    #print(t)
    ldf.at[index,"Lyric Struct"] = t
    x = ''
    for i in range(len(t)):
        x += " " + str(t[i][1]) 
    ldf.at[index,"LS"] = x
print(ldf.loc[0, "Lyric Struct"])    
ldf.head(20)
print(ldf.shape)

C:\Users\lilyg\AppData\Local\Temp/ipykernel_46248/3338202398.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf["Lyric Struct"] = ''
C:\Users\lilyg\AppData\Local\Temp/ipykernel_46248/3338202398.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf["LS"] = ''
C:\Users\lilyg\AppData\Local\Temp/ipykernel_46248/3338202398.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

[('He', 'PRP'), ('said', 'VBD'), ('the', 'DT'), ('way', 'NN'), ('my', 'PRP$'), ('blue', 'JJ'), ('eyes', 'NNS'), ('shined', 'VBD')]
(8358, 29)


In [6]:
df = ldf.copy()

ldf["pattern"] = ""
import numpy
compdf = pd.read_csv("taylor_components.csv")
compdf.shape
#print(compdf.head())

(275, 2)

In [7]:
ptn = []
es = 0
j=0
for i in (range(0, compdf.shape[0])):
    cindex = compdf.iloc[i].name
    elems = compdf.loc[cindex, "elements"]
    e = list(elems.split(', '))
    song = compdf.loc[cindex, "song"]
        
    index = ldf.iloc[i].name
    #print(index)
    
    curr = len(list(ldf.loc[ldf['track_title'] == song ,'pattern']))
    print(song, curr)
    ldf.at[index, 'pattern'] = ""
    p = ""
    for k in range(0, len(e)):
        #linenum = index + k
        #print(k)
        ele = e[k]
       # print(ele)
        ldf.loc[(ldf['track_title'] == song) & (ldf['line'] == k), 'Element'] = str(ele)
        #ldf.at[index, 'Element'] = ele
        if(ele == "Verse 1" or ele == "Verse 2" or ele == "Verse 3" or ele == "Verse"): 
            ldf.loc[(ldf['track_title'] == song) & (ldf['line'] == k), 'pattern'] = "V"
            ldf.loc[(ldf['track_title'] == song) & (ldf['line'] == k), 'e'] = "1"
            p += "V"
        elif(ele == "Chorus" or ele == "Chorus 1" or ele == "Chorus 2" or ele == "Chorus 3"): 
            ldf.loc[(ldf['track_title'] == song) & (ldf['line'] == k), 'pattern'] = "C"
            ldf.loc[(ldf['track_title'] == song) & (ldf['line'] == k), 'e'] = "2"
            p += "C"
        elif(ele == "Outro"): 
            ldf.loc[(ldf['track_title'] == song) & (ldf['line'] == k), 'pattern']= "O"
            ldf.loc[(ldf['track_title'] == song) & (ldf['line'] == k), 'e'] = "3"
            p += "O"
        elif(ele == "Bridge" or ele == "Bridge 1" or ele == "Bridge 2"): 
            ldf.loc[(ldf['track_title'] == song) & (ldf['line'] == k), 'pattern'] = "B"
            ldf.loc[(ldf['track_title'] == song) & (ldf['line'] == k), 'e'] = "4"
            p += "B"
        elif(ele == "Pre-Chorus" or ele == "Pre-Chorus 1" or ele == "Pre-Chorus 2" or ele == "Pre-Chorus 3"):
            ldf.loc[(ldf['track_title'] == song) & (ldf['line'] == k), 'pattern'] = "P"
            ldf.loc[(ldf['track_title'] == song) & (ldf['line'] == k), 'e'] = "5"
            p += "P"
        elif(ele == "Intro"):
            ldf.loc[(ldf['track_title'] == song) & (ldf['line'] == k), 'pattern'] = "I"
            ldf.loc[(ldf['track_title'] == song) & (ldf['line'] == k), 'e'] = "6"
            p += "I"
        else:
            ldf.loc[(ldf['track_title'] == song) & (ldf['line'] == k), 'pattern'] = "X"
            ldf.loc[(ldf['track_title'] == song) & (ldf['line'] == k), 'e'] = "7"
    #print(song, p)
    ptn.append([song, p])
#     es = len(str(e))
#     j += es
    
print(len(ptn))
ldf = ldf.dropna(subset = ["pattern"]) 
print(ldf.head(20) )   


test 0
Tim McGraw 55
Picture To Burn 50
Teardrops On My Guitar 35
A Place In This World 27
Cold As You 22
The Outside 35
Tied Together With A Smile 38
Stay Beautiful 55
Should've Said No 44
Mary's Song (Oh My My My) 0
Our Song 51
I'm Only Me When I'm With You 0
Invisible 48
A Perfectly Good Heart 24
Teardrops On My Guitar (Pop Version) 0
Fearless 0
Fifteen 0
Love Story 0
Hey Stephen 0
White Horse 0
You Belong With Me 0
Breathe 0
Tell Me Why 0
You're Not Sorry 0
The Way I Loved You 0
Forever & Always 0
The Best Day 0
Change 0
Jump Then Fall 0
Untouchable 0
Forever & Always (Piano Version) 0
Come In With The Rain 0
SuperStar 0
The Other Side Of The Door 0
Fearless (Taylor's Version) 0
Fifteen (Taylor's Version) 0
Love Story (Taylor's Version) 0
Hey Stephen (Taylor's Version) 0
White Horse (Taylor's Version) 0
You Belong With Me (Taylor's Version) 0
Breathe (Taylor's Version) 0
Tell Me Why (Taylor's Version) 0
You're Not Sorry (Taylor's Version) 0
The Way I Loved You (Taylor's Version) 0


In [13]:
import numpy as np
ptn = np.asarray(ptn)
#print(ptn)

patterns = {}
for p in range(1, len(ptn)):
    song = ptn[p][0]
    cleaned = ''
    for i in range(len(ptn[p][1])):
        if p == 0:
            cleaned += ptn[p][i] 
        else: 
            if(ptn[p][1][i] == ptn[p][1][i-1]):
                cleaned = cleaned
            else: 
                cleaned += ptn[p][1][i] 
    patterns[song] = cleaned
   # print(ptn[i][0], cleaned)
df = pd.DataFrame(list(patterns.items()),columns = ['song','pattern'])
#df = df[df.pattern != '']
df = df.dropna(subset = ["pattern"])
print(df)

                        song      pattern
0                 Tim McGraw      VCVCBCO
1            Picture To Burn      VCVCBCO
2     Teardrops On My Guitar  VPVPCVPCBCO
3      A Place In This World      VCVCBCO
4                Cold As You       VCVCBC
..                       ...          ...
268       Highway Don't Care     VPVPCPBO
269                 Renegade      VCVCBCO
270  The Joker And The Queen         VCVC
271        Three Sad Virgins    VCVCVCBCO
272   Two Is Better Than One      VCVCBCO

[273 rows x 2 columns]


In [11]:
sdf = pd.read_csv("spotify.csv")
sdf['pattern'] = ''
#print(sdf.head())
print(sdf.shape)
    
for i in df['song'].iteritems():
    #print (i[1])
    p = list(df.loc[(df['song']== i[1]), 'pattern'])[0]
    #print (p)
    sdf.loc[(sdf["track_name"] == i[1]), 'pattern'] = p
    #songp = df.loc[df['song'] == song] 
    
#sdf = sdf[sdf.pattern != '']
sdf = sdf.dropna(subset = ["pattern"]) 
print(sdf.shape)
sdf.head(20)
sdf = sdf.to_csv('spotifywithpatterns.csv')

(269, 23)
(269, 23)


In [14]:
patterns = ["VCVC", "VCVCVC", "VCVCBC", "VPCVPCBC", "VVBV", "VVBVBV", "VCVCVC"]
eqs = [["N","V", "N"], ["N", "V"], ["N", "V", "A"], ["A", "N", "V", "O"]]
parts = {"N": ["NN","NNP", "NNPS", "NNS", "PRP", "PRP$","WDT", "WP"],
        "V": ["VBZ", "VBG", "VBD", "VBP", "VBN"],
        "A" : ["UH", "RS", "RBR","RB", "JJS", "JJR", "JJ", "IN", "DT", "NN"]}

In [15]:
ldf["parts"] = ''
for i in range(ldf.shape[0]):
    ls = ''
    index = ldf.iloc[i].name
    sentence = ldf.loc[index, 'LS']
    sentence = list(sentence.split(" "))
    for i in range(len(sentence)):
        for key,value in parts.items():
            #print(sentence[i])
            if sentence[i] in value:
                ls += key
                print("y")
    ldf.loc[index, "parts"] = ls 
   # print(ls)
ldf.head()
ldf.to_csv("NEWTAYLORCATS.csv")

y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y


y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y


y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y


y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y


y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y


...

y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y


y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y


y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y


y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y


y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
y
